In [1]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json

pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Understanding

* Robert Koch Institute, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

# GITHUB csv data

git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [2]:
data_path='../data/raw/COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [3]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/11/22,6/12/22,6/13/22,6/14/22,6/15/22,6/16/22,6/17/22,6/18/22,6/19/22,6/20/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,180864,180864,181120,181178,181236,181465,181534,181574,181666,181725
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,276690,276731,276731,276821,276821,276821,277141,277141,277409,277444
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265925,265927,265937,265943,265952,265964,265968,265971,265975,265985
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,43224,43224,43224,43224,43449,43449,43449,43449,43449,43449
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761


# Webscrapping

In [4]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
html_table=soup.find('table') # find the table, attention this works if one table exists

In [7]:
all_rows=html_table.find_all('tr')

In [8]:
final_data_list=[]

In [9]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)

In [10]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [11]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,3.768.735,12.330,34.767,"313,1",16.239
3,Bayern,5.039.584,20.283,53.631,"408,1",24.255
4,Berlin,1.079.464,4.332,12.242,"334,1",4.635
5,Branden­burg,807.903,1.876,7.393,"292,1",5.704
6,Bremen,209.549,834,3.702,"544,3",784


# REST API calls


In [12]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [13]:
json_object=json.loads(data.content) 

In [14]:
type(json_object)

dict

In [15]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [16]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])    

In [17]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2910875,15,793319,1655762400000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,27253.626487,4.573731e+10,2.881496e+06,2587,747.438485,21757,1,"747,4",1
1,2,02,Hamburg,Freie und Hansestadt,1852478,6,612388,1655762400000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,33057.774505,2.089396e+09,4.188002e+05,2703,531.396324,9844,3,"531,4",2
2,3,03,Niedersachsen,Land,8003421,9,2513163,1655762400000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,31401.109601,1.299836e+11,4.008988e+06,9560,718.230367,57483,1,"718,2",3
3,4,04,Bremen,Freie Hansestadt,680130,5,209549,1655762400000,04,4132268b-54de-4327-ac1e-760e915112f1,30810.139238,1.119157e+09,3.357177e+05,784,544.307706,3702,1,"544,3",4
4,5,05,Nordrhein-Westfalen,Land,17925570,10,5531269,1655762400000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,30856.865360,8.782936e+10,2.648673e+06,25605,524.312477,93986,7,"524,3",5


In [18]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

In [19]:
pd_full_list.shape[0]

16